In [ ]:
from lhorizon import construct_bulk_horizon_query, fetch_bulk_pointing_table

In [ ]:
# defining position, time span, and so on

# what is the body center of the observing location?
mars_horizons_id = '499'
observer_body_id = mars_horizons_id

# what is the topocentric location relative to that body center?
observer_latitude = 18.4
# note that Horizons always treats east-longitude as
# negative for Mars and other prograde bodies
# *except* the Earth, Moon, and Sun
observer_longitude = -77.5
observer_elevation = 0

# what is the observer looking at?
# (this can also target topocentric sites on the target body,
# but the syntax is somewhat different. obviously this is
# not important in this particular application.)
sun_horizons_id = '10'
target_body_id = sun_horizons_id

# start time, stop time, and observation interval
start = '2020-01-01T00:00:00'
stop = '2023-02-01T00:00:00'
# this can be 's', 'm', 'h', 'd', 'y' for fixed time intervals;
# alternatively, just an integer will give a number of intervals
# equal to (start time - stop_time) / step.
step = "5m"

In [ ]:
# make a bunch of horizons requests at palatable-to-horizons sizes.
# parenthetically, we're actually requesting all the quantities here --
# for whatever reason, horizons isn't particularly faster when you don't,
# so we don't bother restricting at this step.
bulk_query = construct_bulk_horizon_query(
    target_body_id,
    observer_longitude,
    observer_latitude,
    observer_elevation,
    observer_body_id,
    start,
    stop,
    step
)

In [ ]:
%%time
# fetch the data and concatenate it into a pandas dataframe
pointing_table = fetch_bulk_pointing_table(bulk_query)

In [ ]:
# horizons bulk queries don't do well with the 'only when object is visible'
# flags, so we handle it here
daylight_only = pointing_table.loc[pointing_table['alt'] > 0]
# reduce to the quantities of interest
solar_angle_table = daylight_only[['time', 'az', 'alt']]

In [ ]:
# write out to csv
solar_angle_table.to_csv(
    "solar_angle_table " + start + " to " + stop + ".csv",
    index=None
)
